In [2]:
import logging
logging.getLogger('tensorflow').disabled = False

from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
import sys
import numpy as np
import nltk
from importlib import reload
import os

sys.path.append('../src')
from study_design_type.study_type_labels import StudyTypeLabels
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from utilities import excel_reader, excel_writer
from sklearn.utils import shuffle
#import eli5

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
!pip install bert-tensorflow

In [4]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score

def calculate_metrics(results_true, model_results, study_labels):
    average_f1 = 0
    for label in study_labels:
        print(label)
        res_y = [(1 if label in labels else 0) for labels in model_results]
        test_y = [(1 if label in labels else 0) for labels in results_true]
        print(confusion_matrix(test_y, res_y))
        print(classification_report(test_y, res_y))
        f1 = f1_score(test_y, res_y, average="macro")
        print("F1 score: ", f1)
        average_f1 += f1
    print("Average F1: ", average_f1/len(study_labels))

    cnt = len(results_true)
    cnt_precision = 0
    cnt_recall = 0
    cnt_f1 = 0
    cnt_some_not_found = 0
    for i in range(len(results_true)):
        cnt_intersect = len(set(results_true[i]).intersection(set(model_results[i])))
        cnt_correct = len(results_true[i])
        cnt_found = len(model_results[i])
        precision = 0 if cnt_found == 0 else (cnt_intersect/cnt_found)
        recall = 0 if cnt_correct == 0 else (cnt_intersect/cnt_correct)
        f1 = 0
        if (precision + recall) > 0:
            f1 = 2*precision*recall/(precision + recall)
        cnt_precision += precision
        cnt_recall += recall
        cnt_f1 += f1
    print("Precision avg: ", cnt_precision/cnt)
    print("Recall avg: ", cnt_recall/cnt)
    print("F1 avg: ", cnt_f1/cnt)

In [5]:
from sklearn.utils import shuffle

priority_labels = {"No category": 0, "Field study": 0, "Laboratory study": 1, 'Modelling study': 3,
                   'Observational study': 2, 'Review paper': 4, "Economic simulation": 5, "Landscape/environmental simulation": 6}

study_labels = ["Observational study", "Modelling study", "Laboratory study", "Review paper", "Field study",
                "Economic simulation", "Landscape/environmental simulation"]

def create_multi_label_target(df):
    df["label"] = ""
    for i in range(len(df)):
        target_labels = []
        for column in study_labels:
            target_labels.append(df[column].values[i])
        df["label"].values[i] = target_labels
    return df[["title", "abstract", "label"]]

def load_train_study_type_multi_label(folder, files_replication={}):
    all_df = pd.DataFrame()
    for file in os.listdir(folder):
        replicate = 1
        if file in files_replication:
            replicate = files_replication[file]
        print(file, replicate)

        for i in range(replicate):
            train_1 = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder, file))
            train_1 = create_multi_label_target(train_1)
            all_df = pd.concat([all_df, train_1],axis=0)
    print("Size ", len(all_df))
    return all_df

In [21]:
train = load_train_study_type_multi_label("../tmp/new_study_design_type_data/train/", files_replication={
    "df_study_design_type.xlsx": 4,
    "labelled_multi.xlsx": 4,
    "labelled_multi_modelling_JP.xlsx": 4,
    "sample_modelling_more_JP.xlsx": 4,
    "sample_to_label_modelling_study_JP.xlsx": 4,
    "sample_to_label_modelling_21.09_JP.xlsx": 4
})
train = shuffle(train)
valid = load_train_study_type_multi_label("../tmp/new_study_design_type_data/valid/")
test = load_train_study_type_multi_label("../tmp/new_study_design_type_data/test/")

train_keywords_plus_multi.xlsx 1
Read file ../tmp/new_study_design_type_data/train/train_keywords_plus_multi.xlsx: 0.96s
Processed file ../tmp/new_study_design_type_data/train/train_keywords_plus_multi.xlsx: 0.52s
sample_to_label_modelling_study_JP.xlsx 4
Read file ../tmp/new_study_design_type_data/train/sample_to_label_modelling_study_JP.xlsx: 0.01s
Processed file ../tmp/new_study_design_type_data/train/sample_to_label_modelling_study_JP.xlsx: 0.00s
Read file ../tmp/new_study_design_type_data/train/sample_to_label_modelling_study_JP.xlsx: 0.01s
Processed file ../tmp/new_study_design_type_data/train/sample_to_label_modelling_study_JP.xlsx: 0.00s
Read file ../tmp/new_study_design_type_data/train/sample_to_label_modelling_study_JP.xlsx: 0.01s
Processed file ../tmp/new_study_design_type_data/train/sample_to_label_modelling_study_JP.xlsx: 0.00s
Read file ../tmp/new_study_design_type_data/train/sample_to_label_modelling_study_JP.xlsx: 0.01s
Processed file ../tmp/new_study_design_type_data/t

In [9]:
def calculate_labels(df):
    data = np.zeros((len(df), len(study_labels)))
    for i in range(len(df)):
        for j in range(len(study_labels)):
            data[i][j] = df["label"].values[i][j]
    df_data = pd.DataFrame(data, columns=study_labels)
    print(df_data.sum(axis=0))

In [11]:
calculate_labels(train)

Observational study                   8234.0
Modelling study                       3350.0
Laboratory study                      2499.0
Review paper                          3696.0
Field study                           2798.0
Economic simulation                   1409.0
Landscape/environmental simulation    1177.0
dtype: float64


In [18]:
used_texts = set()
for df in [train, valid, test]:
    for i in range(len(df)):
        title = df["title"].values[i].replace(" ", "").strip().lower()
        abstract = df["abstract"].values[i].replace(" ", "").strip().lower()
        used_texts.add((title, abstract))
len(used_texts)

17288

In [19]:
new_data_idx = []
df = excel_reader.ExcelReader().read_df_from_excel("../tmp/processed_cosai/subfolder-0.xlsx")
for i in range(len(df)):
    title = df["title"].values[i].replace(" ", "").strip().lower()
    abstract = df["abstract"].values[i].replace(" ", "").strip().lower()
    if (title, abstract) not in used_texts:
        new_data_idx.append(i)
len(new_data_idx)

Read file ../tmp/processed_cosai/subfolder-0.xlsx: 9.39s
Processed file ../tmp/processed_cosai/subfolder-0.xlsx: 18.52s


29652

In [20]:
data_to_take = df[["title", "abstract"]].take(new_data_idx).copy()

In [21]:
r = _study_type_labeler.predict_with_meta_model(data_to_take[["title", "abstract"]], with_head_tail=True)

INFO:tensorflow:Writing example 0 of 29652


I0921 11:55:34.251390 139922793150272 base_bert_model.py:478] Writing example 0 of 29652


INFO:tensorflow:*** Example ***


I0921 11:55:34.256250 139922793150272 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0921 12:01:32.704144 139922793150272 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0921 12:01:32.833501 139922793150272 session_manager.py:493] Done running local_init_op.


Prepared train from end


In [22]:
for study_type in study_labels:
    data_to_take[study_type] = 0.0
for i in range(len(r[0])):
    results_ind = []
    for idx, column in enumerate(study_labels):
        data_to_take[column].values[i] = r[0][i][idx]

In [23]:
excel_writer.ExcelWriter().save_df_in_excel(data_to_take, "labelled_study_types.xlsx", column_probabilities=study_labels)

Saving...
Saved to labelled_study_types.xlsx


In [22]:
from bert_models import base_bert_model
base_bert_model = reload(base_bert_model)
from bert_models import base_bert_model_meta_model
base_bert_model_meta_model = reload(base_bert_model_meta_model)
from bert_models import base_bert_model_meta_model_with_window_all
base_bert_model_meta_model_with_window_all = reload(base_bert_model_meta_model_with_window_all)
from study_design_type import study_type_labeler
study_type_labeler = reload(study_type_labeler)
from study_design_type import study_type_labeler_window_agg
study_type_labeler_window_agg = reload(study_type_labeler_window_agg)

In [23]:
_study_type_labeler = study_type_labeler.StudyTypeLabeler(
      "/hdd2/data/maryia_pavlovets/study_type_multi_new_data_with_keywords_more_sub_groups_more_data_1",
      max_seq_length = 256, batch_size = 16,
      gpu_device_num_hub = 0, gpu_device_num = 0,
      model_folder = "../model/scibert_scivocab_uncased",
      label_list = list(range(len(study_labels))),
      meta_model_folder = "/hdd2/data/maryia_pavlovets/study_type_multi_meta_new_data_with_keywords_more_sub_groups_more_data_1",
      use_one_layer=True,
      keep_prob=0.8,
      epochs_num=5,
      multilabel=True
      )

Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
INFO:tensorflow:Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/study_type_multi_new_data_with_keywords_more_sub_groups_more_data_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f03624a2e48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0925 13:14:56.835456 139656925230912 estimator.py:201] Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/study_type_multi_new_data_with_keywords_more_sub_groups_more_data_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f03624a2e48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done


In [24]:
_study_type_labeler.train_model(train, valid, use_tails=True)
all_data = pd.concat([train, valid], axis=0)
all_data = shuffle(all_data)
_study_type_labeler.train_meta_model(train, all_data, use_tail=True)

INFO:tensorflow:Writing example 0 of 18218


I0925 13:15:01.170536 139656925230912 base_bert_model.py:478] Writing example 0 of 18218


INFO:tensorflow:*** Example ***


I0925 13:15:01.179352 139656925230912 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:Running local_init_op.


I0925 14:15:47.548761 139656925230912 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0925 14:15:47.643792 139656925230912 session_manager.py:493] Done running local_init_op.


Prepared train from end
Train on 18218 samples, validate on 18900 samples
Epoch 1/5
18218/18218 [==============================] - 1s 80us/sample - loss: 0.0187 - val_loss: 0.0127
Epoch 2/5
18218/18218 [==============================] - 1s 60us/sample - loss: 0.0127 - val_loss: 0.0117
Epoch 3/5
18218/18218 [==============================] - 1s 61us/sample - loss: 0.0111 - val_loss: 0.0112
Epoch 4/5
18218/18218 [==============================] - 1s 60us/sample - loss: 0.0106 - val_loss: 0.0125
Epoch 5/5
18218/18218 [==============================] - 1s 60us/sample - loss: 0.0097 - val_loss: 0.0106


In [25]:
r = _study_type_labeler.predict_with_meta_model(test[["title", "abstract"]], with_head_tail=True)
model_results = []
for i in range(len(r[0])):
    results_ind = []
    for idx, column in enumerate(study_labels):
        if r[0][i][idx] >= 0.5:
            results_ind.append(column)
    if not results_ind:
        results_ind.append("No category")
    model_results.append(results_ind)
full_test = excel_reader.ExcelReader().read_folder("../tmp/new_study_design_type_data/test")
results_true = []
for i in range(len(full_test)):
    results_ind = []
    for column in study_labels:
        if full_test[column].values[i] == 1:
            results_ind.append(column)
    if not results_ind:
        results_ind.append("No category")
    results_true.append(results_ind)

INFO:tensorflow:Writing example 0 of 281


I0925 14:18:11.381800 139656925230912 base_bert_model.py:478] Writing example 0 of 281


INFO:tensorflow:*** Example ***


I0925 14:18:11.386669 139656925230912 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0925 14:18:20.092166 139656925230912 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0925 14:18:20.120461 139656925230912 session_manager.py:493] Done running local_init_op.


Prepared train from end
Read file ../tmp/new_study_design_type_data/test/final_study_type_test_data.xlsx: 0.02s
Processed file ../tmp/new_study_design_type_data/test/final_study_type_test_data.xlsx: 0.01s
Read file ../tmp/new_study_design_type_data/test/sample_modelling_test.xlsx: 0.01s
Processed file ../tmp/new_study_design_type_data/test/sample_modelling_test.xlsx: 0.00s
Read file ../tmp/new_study_design_type_data/test/modelling_test_data_JP.xlsx: 0.01s
Processed file ../tmp/new_study_design_type_data/test/modelling_test_data_JP.xlsx: 0.00s


In [13]:
#with keywords added additional keywords + more data correct labelling
calculate_metrics(results_true, model_results, study_labels)

Observational study
[[160  18]
 [ 27  79]]
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       178
           1       0.81      0.75      0.78       106

   micro avg       0.84      0.84      0.84       284
   macro avg       0.84      0.82      0.83       284
weighted avg       0.84      0.84      0.84       284

F1 score:  0.8275187259599164
Modelling study
[[181  31]
 [ 10  62]]
              precision    recall  f1-score   support

           0       0.95      0.85      0.90       212
           1       0.67      0.86      0.75        72

   micro avg       0.86      0.86      0.86       284
   macro avg       0.81      0.86      0.82       284
weighted avg       0.88      0.86      0.86       284

F1 score:  0.8248890894052185
Laboratory study
[[248   2]
 [ 14  20]]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       250
           1       0.91      0.59      0.71        34



In [16]:
#with keywords added additional keywords + more data other data
calculate_metrics(results_true, model_results, study_labels)

Observational study
[[134  39]
 [ 12  96]]
              precision    recall  f1-score   support

           0       0.92      0.77      0.84       173
           1       0.71      0.89      0.79       108

   micro avg       0.82      0.82      0.82       281
   macro avg       0.81      0.83      0.82       281
weighted avg       0.84      0.82      0.82       281

F1 score:  0.8151244243198266
Modelling study
[[200  12]
 [ 20  49]]
              precision    recall  f1-score   support

           0       0.91      0.94      0.93       212
           1       0.80      0.71      0.75        69

   micro avg       0.89      0.89      0.89       281
   macro avg       0.86      0.83      0.84       281
weighted avg       0.88      0.89      0.88       281

F1 score:  0.8398860398860399
Laboratory study
[[244   4]
 [  6  27]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       248
           1       0.87      0.82      0.84        33



In [26]:
#with keywords added additional keywords + more data other data relabelled
calculate_metrics(results_true, model_results, study_labels)

Observational study
[[136  38]
 [ 14  93]]
              precision    recall  f1-score   support

           0       0.91      0.78      0.84       174
           1       0.71      0.87      0.78       107

   micro avg       0.81      0.81      0.81       281
   macro avg       0.81      0.83      0.81       281
weighted avg       0.83      0.81      0.82       281

F1 score:  0.8105093889407615
Modelling study
[[200  12]
 [ 19  50]]
              precision    recall  f1-score   support

           0       0.91      0.94      0.93       212
           1       0.81      0.72      0.76        69

   micro avg       0.89      0.89      0.89       281
   macro avg       0.86      0.83      0.85       281
weighted avg       0.89      0.89      0.89       281

F1 score:  0.8457165122828147
Laboratory study
[[245   4]
 [  6  26]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       249
           1       0.87      0.81      0.84        32



In [27]:
from bert_models import base_bert_model
base_bert_model = reload(base_bert_model)
base_bert_model_meta_model_with_window_all = reload(base_bert_model_meta_model_with_window_all)
study_type_labeler_window_agg = reload(study_type_labeler_window_agg)
study_type_labeler_window_agg
_study_type_labeler = study_type_labeler_window_agg.StudyTypeLabeler(
      "/hdd2/data/maryia_pavlovets/study_type_multi_new_data_with_keywords_agg_more_sub_groups_more_data_1",
      max_seq_length = 256, batch_size = 16,
      gpu_device_num_hub = 0, gpu_device_num = 0,
      model_folder = "../model/scibert_scivocab_uncased",
      label_list = list(range(len(study_labels))),
      meta_model_folder = "/hdd2/data/maryia_pavlovets/study_type_multi_meta_new_data_with_keywords_agg_more_sub_groups_more_data_1",
      use_one_layer=True,
      keep_prob=0.8,
      epochs_num=5,
      multilabel=True
      )

Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
INFO:tensorflow:Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/study_type_multi_new_data_with_keywords_agg_more_sub_groups_more_data_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f032f23b828>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0925 14:18:23.640690 139656925230912 estimator.py:201] Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/study_type_multi_new_data_with_keywords_agg_more_sub_groups_more_data_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f032f23b828>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done


In [28]:
_study_type_labeler.train_model(train, valid, use_tails=True)
all_data = pd.concat([train, valid], axis=0)
all_data = shuffle(all_data)
_study_type_labeler.train_meta_model(train, all_data, use_tail=True)

INFO:tensorflow:Writing example 0 of 18218


I0925 14:18:23.972902 139656925230912 base_bert_model.py:478] Writing example 0 of 18218


INFO:tensorflow:*** Example ***


I0925 14:18:23.979646 139656925230912 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:Running local_init_op.


I0925 15:33:28.927693 139656925230912 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0925 15:33:29.024277 139656925230912 session_manager.py:493] Done running local_init_op.


Train on 18218 samples, validate on 18900 samples
Epoch 1/5
18218/18218 [==============================] - 2s 94us/sample - loss: 0.0191 - val_loss: 0.0134
Epoch 2/5
18218/18218 [==============================] - 1s 62us/sample - loss: 0.0121 - val_loss: 0.0104
Epoch 3/5
18218/18218 [==============================] - 1s 62us/sample - loss: 0.0111 - val_loss: 0.0089
Epoch 4/5
18218/18218 [==============================] - 1s 63us/sample - loss: 0.0097 - val_loss: 0.0089
Epoch 5/5
18218/18218 [==============================] - 1s 62us/sample - loss: 0.0093 - val_loss: 0.0087


In [29]:
r = _study_type_labeler.predict_with_meta_model(test[["title", "abstract"]], with_head_tail=True)
model_results = []
for i in range(len(r[0])):
    results_ind = []
    for idx, column in enumerate(study_labels):
        if r[0][i][idx] >= 0.5:
            results_ind.append(column)
    if not results_ind:
        results_ind.append("No category")
    model_results.append(results_ind)
full_test = excel_reader.ExcelReader().read_folder("../tmp/new_study_design_type_data/test")
results_true = []
for i in range(len(full_test)):
    results_ind = []
    for column in study_labels:
        if full_test[column].values[i] == 1:
            results_ind.append(column)
    if not results_ind:
        results_ind.append("No category")
    results_true.append(results_ind)

INFO:tensorflow:Writing example 0 of 281


I0925 15:35:53.378374 139656925230912 base_bert_model.py:478] Writing example 0 of 281


INFO:tensorflow:*** Example ***


I0925 15:35:53.384367 139656925230912 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0925 15:36:10.002683 139656925230912 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0925 15:36:10.038602 139656925230912 session_manager.py:493] Done running local_init_op.


Read file ../tmp/new_study_design_type_data/test/final_study_type_test_data.xlsx: 0.02s
Processed file ../tmp/new_study_design_type_data/test/final_study_type_test_data.xlsx: 0.01s
Read file ../tmp/new_study_design_type_data/test/sample_modelling_test.xlsx: 0.01s
Processed file ../tmp/new_study_design_type_data/test/sample_modelling_test.xlsx: 0.00s
Read file ../tmp/new_study_design_type_data/test/modelling_test_data_JP.xlsx: 0.01s
Processed file ../tmp/new_study_design_type_data/test/modelling_test_data_JP.xlsx: 0.00s


In [17]:
# with keywords  agg + more data correct labelling
calculate_metrics(results_true, model_results, study_labels)

Observational study
[[163  15]
 [ 28  78]]
              precision    recall  f1-score   support

           0       0.85      0.92      0.88       178
           1       0.84      0.74      0.78       106

   micro avg       0.85      0.85      0.85       284
   macro avg       0.85      0.83      0.83       284
weighted avg       0.85      0.85      0.85       284

F1 score:  0.8336942163391483
Modelling study
[[190  22]
 [  8  64]]
              precision    recall  f1-score   support

           0       0.96      0.90      0.93       212
           1       0.74      0.89      0.81        72

   micro avg       0.89      0.89      0.89       284
   macro avg       0.85      0.89      0.87       284
weighted avg       0.90      0.89      0.90       284

F1 score:  0.8684779252855819
Laboratory study
[[248   2]
 [ 10  24]]
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       250
           1       0.92      0.71      0.80        34



In [20]:
# with keywords  agg + more data other data
calculate_metrics(results_true, model_results, study_labels)

Observational study
[[139  34]
 [ 16  92]]
              precision    recall  f1-score   support

           0       0.90      0.80      0.85       173
           1       0.73      0.85      0.79       108

   micro avg       0.82      0.82      0.82       281
   macro avg       0.81      0.83      0.82       281
weighted avg       0.83      0.82      0.82       281

F1 score:  0.8169428809672712
Modelling study
[[199  13]
 [ 18  51]]
              precision    recall  f1-score   support

           0       0.92      0.94      0.93       212
           1       0.80      0.74      0.77        69

   micro avg       0.89      0.89      0.89       281
   macro avg       0.86      0.84      0.85       281
weighted avg       0.89      0.89      0.89       281

F1 score:  0.8473281104860052
Laboratory study
[[245   3]
 [  7  26]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       248
           1       0.90      0.79      0.84        33



In [30]:
# with keywords  agg + more data other data relabelled
calculate_metrics(results_true, model_results, study_labels)

Observational study
[[145  29]
 [ 13  94]]
              precision    recall  f1-score   support

           0       0.92      0.83      0.87       174
           1       0.76      0.88      0.82       107

   micro avg       0.85      0.85      0.85       281
   macro avg       0.84      0.86      0.85       281
weighted avg       0.86      0.85      0.85       281

F1 score:  0.8454426401257202
Modelling study
[[197  15]
 [ 15  54]]
              precision    recall  f1-score   support

           0       0.93      0.93      0.93       212
           1       0.78      0.78      0.78        69

   micro avg       0.89      0.89      0.89       281
   macro avg       0.86      0.86      0.86       281
weighted avg       0.89      0.89      0.89       281

F1 score:  0.8559269893355209
Laboratory study
[[247   2]
 [  9  23]]
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       249
           1       0.92      0.72      0.81        32



In [3]:
def predict_proba(doc):
    return np.asarray(_study_type_labeler.predict_with_meta_model(pd.DataFrame([("title", d) for d in doc], columns=["title", "abstract"]), with_head_tail = True, recreate_model = True)[0])

In [4]:
idx = 15
text = test_plus_stefan["abstract"].values[idx]
for column in ['Observational study', 'Modeling study', 'Laboratory study', 'Review paper', 'Field study']:
    if test_plus_stefan[column].values[idx] > 0.5:
        print(column)

Observational study
Modeling study


In [5]:
from bert_models import base_bert_model_meta_model
base_bert_model_meta_model = reload(base_bert_model_meta_model)
from study_design_type import study_type_labeler
study_type_labeler = reload(study_type_labeler)
import eli5
from eli5.lime import TextExplainer

label = "Observational study"
_study_type_labeler = study_type_labeler.StudyTypeLabeler(
            "/hdd2/data/maryia_pavlovets/study_type_exps/study_type_with_tails_%s"%label,
            max_seq_length = 256, batch_size = 16,
          gpu_device_num_hub = 0, gpu_device_num = 0,
          model_folder = "../model/scibert_scivocab_uncased",
          label_list = [0,1],
          meta_model_folder = "/hdd2/data/maryia_pavlovets/study_type_exps/meta_model_%s_2layer_0.8"%label,
          use_one_layer=False,
          keep_prob=0.8,
          epochs_num=8
          )
te = TextExplainer(random_state=42)
te.fit(text, predict_proba)
te.show_prediction(target_names=[0,1])

Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Used for model gpu 0


In [6]:
_study_type_labeler.generate_test_file("../notebooks/study_type_all_left_dataset_5_filtered.xlsx","new_sample_for_test.xlsx",size=80, withBoost=True,use_tail=True)

5    45482
1    18173
0     6057
3     1569
4      135
2       43
Name: label, dtype: int64
Used for model gpu 2

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Used for model gpu 2
Used for model gpu 2
Saving...
Saved to new_sample_for_test.xlsx
Saving...
Saved to ../notebooks/study_type_all_left_dataset_5_filtered.xlsx


In [ ]:
from text_processing import search_engine_insensitive_to_spelling
from text_processing import all_column_filler
from text_processing import abbreviations_resolver

_abbreviations_resolver = abbreviations_resolver.AbbreviationsResolver([])
_abbreviations_resolver.load_model("../model/abbreviations_dicts")

folder_with_processed = "/hdd2/data/maryia_pavlovets/processed_cosai_deduplicated_with_study_type"
os.makedirs(folder_with_processed, exist_ok=True)
for file in os.listdir("../tmp/processed_deduplicated_cosai_processed"):
    df = excel_reader.ExcelReader().read_df_from_excel(os.path.join("../tmp/processed_deduplicated_cosai_processed", file))
    file_part = file.split(".")[0]
    search_engine_inverted_index = search_engine_insensitive_to_spelling.SearchEngineInsensitiveToSpelling(
        load_abbreviations = True)
    if os.path.exists(folder_with_search_indices + file_part):
        search_engine_inverted_index.load_model(folder_with_search_indices + file_part)
    else:
        search_engine_inverted_index.create_inverted_index(df)
        search_engine_inverted_index.save_model(folder_with_search_indices + file_part)
    
    _all_column_filler = all_column_filler.AllColumnFiller()
    df = _all_column_filler.fill_columns_for_df(
        df, search_engine_inverted_index, _abbreviations_resolver, settings_json = {"columns":[
            {"column_filler_class": "StudyTypeLabeller",
             "folder": "/hdd2/data/maryia_pavlovets/study_type_multi_new_data_with_keywords_agg_more_sub_groups_more_data_1",
             "meta_folder": "/hdd2/data/maryia_pavlovets/study_type_multi_meta_new_data_with_keywords_agg_more_sub_groups_more_data_1",
             "scibert_model_folder": "../model/scibert_scivocab_uncased", "model_with_agg": True},
        ]})
    excel_writer.ExcelWriter().save_df_in_excel(df, os.path.join(folder_with_processed, file))

/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ._gradient_boosting import predict_stages
/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Read file ../tmp/processed_deduplicated_cosai_processed/data_year_2004.xlsx: 9.84s
Processed file ../tmp/processed_deduplicated_cosai_processed/data_year_2004.xlsx: 18.75s
Started processing  {'column_filler_class': 'StudyTypeLabeller', 'folder': '/hdd2/data/maryia_pavlovets/study_type_multi_new_data_with_keywords_agg_more_sub_groups_more_data_1', 'meta_folder': '/hdd2/data/maryia_pavlovets/study_type_multi_meta_new_data_with_keywords_agg_more_sub_groups_more_data_1', 'scibert_model_folder': '../model/scibert_scivocab_uncased'}
High level label: Book chapter
High level label: Field study
High level label: Greenhouse study
High level label: Impact evaluation
High level label: Laboratory study
High level label: Meta analysis
High level label: Modeling study
High level label: Observational study
High level label: Randomized controlled trials
High level label: Review paper
High level label: Systematic

I0925 18:00:12.921892 139870121596736 estimator.py:201] Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/study_type_multi_new_data_with_keywords_agg_more_sub_groups_more_data_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3483ffb860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done
INFO:tensorflow:Writing example 0 of 30265


I0925 18:00:12.965277 139870121596736 base_bert_model.py:478] Writing example 0 of 30265


...

I0925 20:32:48.808609 139870121596736 base_bert_model.py:459] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


I0925 20:32:48.809087 139870121596736 base_bert_model.py:460] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:label: [0, 0, 0, 0, 0, 0, 0] (id = [0, 0, 0, 0, 0, 0, 0])


I0925 20:32:48.809512 139870121596736 base_bert_model.py:461] label: [0, 0, 0, 0, 0, 0, 0] (id = [0, 0, 0, 0, 0, 0, 0])
